In [17]:
import cv2
import pickle


#get the size of the box for one car parking
width, height = 107, 48
 
try:
    with open('CarParkPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []
 
#creating parking spots with mouse click 
def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:        posList.append((x, y))
    if events == cv2.EVENT_RBUTTONDOWN:
        for i, pos in enumerate(posList):
            x1, y1 = pos
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(i)
                
    #save the clicks 
    with open('CarParkPos', 'wb') as f:
        pickle.dump(posList, f)
 
 
while True:
    img = cv2.imread('carParkImg.png')
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)
 
    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)
    cv2.waitKey(1)


SyntaxError: invalid syntax (89994941.py, line 27)

In [59]:
import cv2
import pickle
import cvzone
import numpy as np
import datetime
import pandas as pd
import json
from flask import Flask



# Video feed
cap = cv2.VideoCapture('carPark.mp4')
 
with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)
 
width, height = 107, 48
 
# detect if a spot is empty or not based on the the amount of white color 
def checkParkingSpace(imgPro):
    spaceCounter = 0
 
    for pos in posList:
        x, y = pos
 
        imgCrop = imgPro[y:y + height, x:x + width]
        # cv2.imshow(str(x * y), imgCrop)
        count = cv2.countNonZero(imgCrop)
 
 
        if count < 900:
            color = (0, 255, 0)
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2
 
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                       	thickness=2, offset=0, colorR=color)
 
    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (100, 50), scale=3,
                       	thickness=5, offset=20, colorR=(0,200,0))
    return spaceCounter
timeMostCrowded = len(posList)
timeLessCrowded = 0
timeList=[]
availableSpotList=[]


while True:
 
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    success, img = cap.read()
    if success == True:
        
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)
        imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 25, 16)
        imgMedian = cv2.medianBlur(imgThreshold, 5)
        kernel = np.ones((3, 3), np.uint8)
        imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)
        ct = datetime.datetime.now()
        time = str(ct.minute)
        # time = str(ct.hour) +"and"+ str(ct.minute)+"mins"
        timeList.append(time )
        availableSpot.append(checkParkingSpace(imgDilate))
        
        # get the maximum and minimum count of open spots
        if (checkParkingSpace(imgDilate) < timeMostCrowded):
            timeMostCrowded = (checkParkingSpace(imgDilate))
        if (checkParkingSpace(imgDilate) > timeLessCrowded):
            timeLessCrowded = (checkParkingSpace(imgDilate))

    
        #print(checkParkingSpace(imgDilate), ct)   
        cv2.imshow("Image", img)
        # cv2.imshow("ImageBlur", imgBlur)
        # cv2.imshow("ImageThres", imgMedian)
        # cv2.waitKey(10)
        if cv2.waitKey(1) == ord('q'):
            
            break
    #else:
# dicts = {}

# for i in range(len(timeList)):
#     dicts[timeList[i]] = availableSpot[i]
# print(dicts)   
# print(timeList)
# print(availableSpot)

print("Most crowded today with ", timeMostCrowded, "open spots")
print("Less crowded today at with ",  timeLessCrowded, "open spots")
print("currently",checkParkingSpace(imgDilate),"spots are available") 
cap.release()
cv2.destroyAllWindows()

    

    



Most crowded today with  12 open spots
Less crowded today at with  13 open spots
currently 13 spots are available
